In [3]:
# -*- coding: utf-8 -*-
"""
Exploration of TARIFES.csv

This notebook explores the TARIFES.csv file, which contains details about the 
different parking tariffs applicable to surface parking spots in Barcelona.
"""

import pandas as pd
import os

In [4]:
# --- Configuration ---
DATA_DIR = "../data/raw" # Adjust if your notebook structure is different
TARIFES_FILE = os.path.join(DATA_DIR, "TARIFES.csv")


In [5]:
# --- Load Data ---
print(f"Loading data from: {TARIFES_FILE}")
try:
    df_tarifes = pd.read_csv(TARIFES_FILE)
    print("Data loaded successfully.")
except FileNotFoundError:
    print(f"Error: File not found at {TARIFES_FILE}")
    print("Please ensure the data ingestion script has run successfully.")
    df_tarifes = pd.DataFrame() # Create empty df to avoid subsequent errors


Loading data from: ../data/raw/TARIFES.csv
Data loaded successfully.


In [6]:
# --- Initial Inspection ---
if not df_tarifes.empty:
    print("\n--- First 5 Rows ---")
    # Increase display width temporarily for potentially long descriptions
    with pd.option_context('display.max_colwidth', 100):
        print(df_tarifes.head())

    print("\n--- DataFrame Info ---")
    df_tarifes.info()

    print("\n--- Descriptive Statistics ---")
    # Include 'all' to get stats for non-numeric columns too
    print(df_tarifes.describe(include='all'))

    print("\n--- Missing Values per Column ---")
    print(df_tarifes.isnull().sum())

    print("\n--- Unique Values in Key Columns ---")
    # Check ID_TARIFA as it's the primary key we expect
    if 'ID_TARIFA' in df_tarifes.columns:
        print(f"Unique ID_TARIFA values: {df_tarifes['ID_TARIFA'].nunique()}")
        print(f"Total rows: {len(df_tarifes)}")
        # If the number of unique IDs matches the total rows, ID_TARIFA is likely a unique identifier
        if df_tarifes['ID_TARIFA'].nunique() == len(df_tarifes):
             print("ID_TARIFA appears to be a unique identifier for each tariff.")
        else:
             print("WARN: ID_TARIFA might have duplicate values.")
             # print(df_tarifes[df_tarifes.duplicated(subset=['ID_TARIFA'], keep=False)])
    else:
        print("Column 'ID_TARIFA' not found.")

    # Check unique values in code/description columns
    if 'CODI_TARIFA' in df_tarifes.columns:
        print(f"Unique CODI_TARIFA values: {df_tarifes['CODI_TARIFA'].nunique()}")
        print(df_tarifes['CODI_TARIFA'].value_counts())
    if 'DESCRIPCIO' in df_tarifes.columns:
        print(f"\nUnique DESCRIPCIO values: {df_tarifes['DESCRIPCIO'].nunique()}")
    if 'DESC_CURTA' in df_tarifes.columns:
        print(f"\nUnique DESC_CURTA values: {df_tarifes['DESC_CURTA'].nunique()}")
        # print(df_tarifes['DESC_CURTA'].unique()[:20]) # Print sample if needed

else:
    print("\nDataFrame is empty, cannot perform inspection.")


# --- TODO: Further Analysis ---
# - Examine the different CODI_TARIFA values (A, B, C, RES, DUM etc.) and their meanings.
# - Analyze the pricing columns (IMPORT_FRACCIO, IMPORT_MAXIM) - check data types, distributions.
# - Analyze the time limit columns (TEMPS_MINIM, TEMPS_MAXIM) - check data types, units (minutes?).
# - Investigate the 'UTILITARI_PRO' and '_id' columns.
# - Clean data types (e.g., convert numeric columns currently stored as objects).
# - Check for correlations between columns.

print("\n--- End of Initial Exploration ---")



--- First 5 Rows ---
   ID_TARIFA  ID_TIPUS_CONFIGURACIO  ID_TIPUS_TARIFA CODI_TARIFA  \
0         46                      1                1           A   
1         47                      1                1           B   
2         48                      1                1           C   
3         49                      1                1           D   
4         52                      2                2         RES   

   TIPUS_FRACCIO  TEMPS_MAXIM  TEMPS_MINIM  \
0              1            2            0   
1              1            2            0   
2              1            3            0   
3              1            4            0   
4              2           20            1   

                                        DESCRIPCIO       DESC_CURTA  \
0            2,50 euros/hora Temp.max Est. 2 hores   2,50 E/h <= 2h   
1            2,25 euros/hora Temp.max Est. 2 hores   2,25 E/h <= 2h   
2            1,96 euros/hora Temp.max Est. 3 hores   1,96 E/h <= 3h   
3       

In [7]:
# --- Further Analysis ---

import numpy as np

print("\n--- Analyzing CODI_TARIFA ---")
# Display all unique codes again for reference
print("Unique CODI_TARIFA values:")
print(np.sort(df_tarifes['CODI_TARIFA'].unique()))

# Let's look at rows grouped by common prefixes/types inferred from codes
print("\n--- Sample Rows by CODI_TARIFA Type ---")
codes_to_show = ['A', 'B', 'C', 'D', 'RES', 'GRA1', 'GRB1', 'BU1', 'MA1', 'EDM1', 'CYD']
# Increase display width
with pd.option_context('display.max_colwidth', 120, 'display.width', 150):
     for code in codes_to_show:
         sample = df_tarifes[df_tarifes['CODI_TARIFA'] == code]
         if not sample.empty:
             print(f"\n--- Sample for CODI_TARIFA = {code} ---")
             print(sample)
         else:
             print(f"\nCode {code} not found for sampling.")


print("\n--- Analyzing TIPUS_FRACCIO ---")
print("Value counts for TIPUS_FRACCIO:")
print(df_tarifes['TIPUS_FRACCIO'].value_counts())
# Display rows for each TIPUS_FRACCIO value to infer meaning
with pd.option_context('display.max_colwidth', 100):
    for val in sorted(df_tarifes['TIPUS_FRACCIO'].unique()):
        print(f"\n--- Rows where TIPUS_FRACCIO == {val} ---")
        print(df_tarifes[df_tarifes['TIPUS_FRACCIO'] == val][['CODI_TARIFA', 'DESCRIPCIO', 'TEMPS_MAXIM', 'IMPORT_FRACCIO']].head())
# Tentative conclusion: 1 = Hourly rate?, 2 = Daily rate (RES)?, 0 = Free/Special?

print("\n--- Analyzing Time Columns (TEMPS_MAXIM / TEMPS_MINIM) ---")
print("Summary stats for TEMPS_MAXIM:")
print(df_tarifes['TEMPS_MAXIM'].describe())
print("\nValue counts for TEMPS_MAXIM:")
print(df_tarifes['TEMPS_MAXIM'].value_counts().sort_index())

print("\nSummary stats for TEMPS_MINIM:")
print(df_tarifes['TEMPS_MINIM'].describe())
print("\nValue counts for TEMPS_MINIM:")
print(df_tarifes['TEMPS_MINIM'].value_counts().sort_index())

# Check units based on description samples
print("\n--- Time Units Check (based on descriptions) ---")
# Look at descriptions for different TEMPS_MAXIM values
with pd.option_context('display.max_colwidth', 100):
    for time_val in [1, 2, 4, 20, 30]: # Sample different max times
         sample = df_tarifes[df_tarifes['TEMPS_MAXIM'] == time_val]
         if not sample.empty:
             print(f"\nDescriptions for TEMPS_MAXIM == {time_val}:")
             print(sample[['CODI_TARIFA', 'DESCRIPCIO']].head())
# Conclusion: Seems like hours for A/B/C/D/BU/MA/MB, days for RES, maybe minutes for DUM (e.g., 30 -> 30 mins?)

print("\n--- Analyzing Pricing Columns (IMPORT_FRACCIO / IMPORT_MAXIM) ---")
# Check relationship: Does IMPORT_MAXIM often equal IMPORT_FRACCIO * TEMPS_MAXIM?
# NOTE: This calculation assumes TEMPS_MAXIM is in the same unit as the fraction rate (e.g., hours)
# We need to be careful with units (e.g., 'RES' is per day)

# Calculate expected max cost where applicable (assuming hourly)
# We filter out RES (daily) and potentially others where units mismatch
hourly_rate_codes = df_tarifes[~df_tarifes['CODI_TARIFA'].isin(['RES']) & (df_tarifes['TIPUS_FRACCIO'] == 1)]
calculated_max = hourly_rate_codes['IMPORT_FRACCIO'] * hourly_rate_codes['TEMPS_MAXIM']
discrepancy = calculated_max - hourly_rate_codes['IMPORT_MAXIM']

print("Checking consistency: IMPORT_MAXIM vs (IMPORT_FRACCIO * TEMPS_MAXIM) [assuming hourly rate]")
print("Number of rows checked (hourly assumed):", len(hourly_rate_codes))
print("Discrepancy summary (Calculated - Actual):")
print(discrepancy.describe())
# Look at cases with large discrepancies
large_diff = hourly_rate_codes[discrepancy.abs() > 0.01] # Allowing for floating point minor diffs
if not large_diff.empty:
    print("\nTariffs with notable difference between Actual and Calculated IMPORT_MAXIM:")
    with pd.option_context('display.max_colwidth', 100):
        print(large_diff[['CODI_TARIFA', 'DESCRIPCIO', 'TEMPS_MAXIM', 'IMPORT_FRACCIO', 'IMPORT_MAXIM']])
else:
    print("\nIMPORT_MAXIM generally consistent with IMPORT_FRACCIO * TEMPS_MAXIM (for assumed hourly rates).")

print("\n--- Checking for Duplicate DESCRIPCIO ---")
duplicated_descriptions = df_tarifes[df_tarifes.duplicated(subset=['DESCRIPCIO'], keep=False)]
if not duplicated_descriptions.empty:
    print(f"Found {duplicated_descriptions['DESCRIPCIO'].nunique()} descriptions shared by multiple Tariffs.")
    # Sort by description to group them together for easier comparison
    with pd.option_context('display.max_colwidth', 100, 'display.width', 150):
         print(duplicated_descriptions.sort_values(by=['DESCRIPCIO', 'ID_TARIFA']))
else:
    print("No duplicate descriptions found across different ID_TARIFA.")


# --- TODO: Next Steps ---
# - Create a mapping or function to interpret CODI_TARIFA (e.g., 'A' -> Blue Zone A).
# - Create a function to calculate parking cost based on duration, ID_TARIFA, potentially using 
#   IMPORT_FRACCIO, IMPORT_MAXIM, TEMPS_MAXIM, TEMPS_MINIM and TIPUS_FRACCIO.
# - Clarify the meaning of ID_TIPUS_CONFIGURACIO/ID_TIPUS_TARIFA if possible (might require external lookup).

print("\n--- End of Further Analysis ---")


--- Analyzing CODI_TARIFA ---
Unique CODI_TARIFA values:
['A' 'B' 'BU1' 'BU2' 'BU3' 'BU4' 'BUA1' 'BUA2' 'BUAV' 'BUB3' 'BUB4' 'BUFM'
 'C' 'CYD' 'D' 'EDH1' 'EDM1' 'EDM2' 'GRA1' 'GRA2' 'GRB1' 'GRB2' 'MA1'
 'MA2' 'MB1' 'MB2' 'RES']

--- Sample Rows by CODI_TARIFA Type ---

--- Sample for CODI_TARIFA = A ---
   ID_TARIFA  ID_TIPUS_CONFIGURACIO  ID_TIPUS_TARIFA CODI_TARIFA  TIPUS_FRACCIO  TEMPS_MAXIM  TEMPS_MINIM                             DESCRIPCIO  \
0         46                      1                1           A              1            2            0  2,50 euros/hora Temp.max Est. 2 hores   

       DESC_CURTA  IMPORT_FRACCIO  IMPORT_MAXIM  
0  2,50 E/h <= 2h             2.5           5.0  

--- Sample for CODI_TARIFA = B ---
   ID_TARIFA  ID_TIPUS_CONFIGURACIO  ID_TIPUS_TARIFA CODI_TARIFA  TIPUS_FRACCIO  TEMPS_MAXIM  TEMPS_MINIM                             DESCRIPCIO  \
1         47                      1                1           B              1            2            0  2,25 e

In [8]:
# --- Structure Tariff Information ---

import math

print("\n--- Structuring Tariff Information ---")

# 1. Create Tariff Type Mapping (based on CODI_TARIFA prefixes/values)
def map_tariff_type(code):
    if code in ['A', 'B', 'C', 'D']:
        return f"Blue Zone {code}"
    elif code == 'RES':
        return "Resident"
    elif code.startswith('GR'):
        return "Green Zone" # Could refine GRA vs GRB if meaning known
    elif code.startswith('BU'):
        if code == 'BUFM': return "Bus Function/Event"
        return "Bus Zone" # Could refine BU vs BUA etc.
    elif code.startswith('MA') or code.startswith('MB'):
        return "Motorcycle"
    elif code.startswith('ED'):
        return "School/Special Zone"
    elif code == 'CYD':
        return "Loading/Unloading (DUM)"
    else:
        return "Unknown"

df_tarifes['TARIFA_TYPE'] = df_tarifes['CODI_TARIFA'].apply(map_tariff_type)
print("Added 'TARIFA_TYPE' column.")
print(df_tarifes[['CODI_TARIFA', 'TARIFA_TYPE']].value_counts())

# 2. Add Time Unit Column
def map_time_unit(row):
    code = row['CODI_TARIFA']
    tipus_fraccio = row['TIPUS_FRACCIO']
    temps_max = row['TEMPS_MAXIM'] # Use TEMPS_MAXIM as hint too

    if code == 'RES': return 'days'
    if code == 'CYD' or code.startswith('ED'): return 'minutes'
    if tipus_fraccio == 1: return 'hours' # Most hourly rates
    if code == 'BUFM': return 'event/hours' # Special case, likely hours (24)
    # Fallback based on typical values if tipus_fraccio is ambiguous (e.g., 0 or 2)
    if temps_max < 5: return 'hours' # Usually 1-4 hours max
    if temps_max > 4 and temps_max <= 30: return 'minutes' # Usually 10-30 mins max

    return 'unknown' # Fallback

df_tarifes['TIME_UNIT'] = df_tarifes.apply(map_time_unit, axis=1)
print("\nAdded 'TIME_UNIT' column.")
print(df_tarifes[['TARIFA_TYPE', 'TIME_UNIT', 'TIPUS_FRACCIO']].value_counts())


# 3. Create Tariff Details Function using a dictionary lookup
# Create the dictionary first for efficient lookup
tariff_details_dict = {}
# Convert relevant columns to basic python types for storage in dict
for index, row in df_tarifes.iterrows():
    tariff_id = int(row['ID_TARIFA']) # Ensure key is python int
    tariff_details_dict[tariff_id] = {
        'code': row['CODI_TARIFA'],
        'type': row['TARIFA_TYPE'],
        'desc': row['DESC_CURTA'],
        'frac_type': int(row['TIPUS_FRACCIO']),
        'max_time': float(row['TEMPS_MAXIM']), # Use float for consistency
        'min_time': float(row['TEMPS_MINIM']),
        'time_unit': row['TIME_UNIT'],
        'rate': float(row['IMPORT_FRACCIO']), # Rate per fraction (hour/day/event)
        'max_cost': float(row['IMPORT_MAXIM'])
    }

def get_tariff_details(tariff_id):
    """
    Returns a dictionary with structured details for a given tariff ID.
    """
    return tariff_details_dict.get(tariff_id, None)


print("\n--- Example Usage of get_tariff_details ---")
test_ids = df_tarifes.sample(5)['ID_TARIFA'].tolist()
if 46 not in test_ids: test_ids.append(46) # Blue A
if 52 not in test_ids: test_ids.append(52) # RES
if 58 not in test_ids: test_ids.append(58) # DUM (CYD)

print(f"Testing with Tariff IDs: {test_ids}")

for tid in test_ids:
    details = get_tariff_details(tid)
    print(f"\n--- Details for ID_TARIFA = {tid} ---")
    if details:
        for key, value in details.items():
            print(f"  {key}: {value}")
    else:
        print("  Details not found.")

# --- TODO: Next Steps ---
# - Create a function `calculate_parking_cost(tariff_id, duration_minutes)`
#   - This function will use get_tariff_details.
#   - Needs to convert input duration_minutes to the correct unit based on 'time_unit'.
#   - Needs to handle different 'frac_type' (hourly, daily, fixed).
#   - Needs to respect 'min_time', 'max_time', and 'max_cost'.

print("\n--- End of Tariff Structuring ---")


--- Structuring Tariff Information ---
Added 'TARIFA_TYPE' column.
CODI_TARIFA  TARIFA_TYPE            
A            Blue Zone A                1
D            Blue Zone D                1
MB2          Motorcycle                 1
MB1          Motorcycle                 1
MA2          Motorcycle                 1
MA1          Motorcycle                 1
GRB2         Green Zone                 1
GRB1         Green Zone                 1
GRA2         Green Zone                 1
GRA1         Green Zone                 1
EDM2         School/Special Zone        1
EDM1         School/Special Zone        1
EDH1         School/Special Zone        1
CYD          Loading/Unloading (DUM)    1
B            Blue Zone B                1
C            Blue Zone C                1
BUFM         Bus Function/Event         1
BUB4         Bus Zone                   1
BUB3         Bus Zone                   1
BUAV         Bus Zone                   1
BUA2         Bus Zone                   1
BUA1         

In [9]:
# --- Create Cost Calculation Function ---

import math
import numpy as np

print("\n--- Defining Parking Cost Calculation Function ---")

def calculate_parking_cost(tariff_id, duration_minutes):
    """
    Calculates the estimated parking cost for a given tariff and duration.

    Args:
        tariff_id (int): The ID_TARIFA.
        duration_minutes (float): The parking duration in minutes.

    Returns:
        float: The calculated parking cost, or numpy.nan if calculation fails.
    """
    details = get_tariff_details(tariff_id)
    if not details:
        print(f"Warning: Tariff ID {tariff_id} not found.")
        return np.nan # Return NaN for unknown tariff

    if duration_minutes < 0:
         return 0.0 # Negative duration makes no sense, assume zero cost

    cost = 0.0
    unit = details['time_unit']
    rate = details['rate']
    max_time = details['max_time']
    min_time = details['min_time'] # Note: unit depends on tariff type
    max_cost = details['max_cost']
    frac_type = details['frac_type']

    # --- Convert duration to the tariff's unit ---
    duration_in_tariff_units = 0
    if unit == 'hours':
        # Convert duration to hours, handle potential division by zero if rate is per hour but max_time is 0? Unlikely
        duration_in_tariff_units = duration_minutes / 60.0
        # Apply min_time check (assume min_time is also in hours if unit is hours)
        if duration_in_tariff_units < min_time:
             duration_in_tariff_units = min_time # Charge for minimum time if applicable? Or return 0? Check rules. Let's assume charge min.
        # Apply max_time check
        duration_in_tariff_units = min(duration_in_tariff_units, max_time)

    elif unit == 'days':
        # Convert duration to days (rounding up, as partial days usually count as full)
        duration_in_tariff_units = math.ceil(duration_minutes / (60.0 * 24.0))
        # Apply min_time check (assume min_time is in days for RES) - ID 52 has min_time 1 day
        if duration_in_tariff_units < min_time:
             duration_in_tariff_units = min_time
        # Apply max_time check
        duration_in_tariff_units = min(duration_in_tariff_units, max_time)

    elif unit == 'minutes':
        duration_in_tariff_units = duration_minutes
        # Apply min_time check (assume minutes)
        if duration_in_tariff_units < min_time:
            duration_in_tariff_units = min_time # Or maybe return 0 if less than min? For DUM, likely 0 cost anyway.
        # Apply max_time check
        duration_in_tariff_units = min(duration_in_tariff_units, max_time)
        
    elif unit == 'event/hours': # BUFM - Special case
         # Assume rate is per event (function), max_cost likely applies. Max_time might be hours?
         # Simple approach: If any duration > 0, charge the rate, up to max_cost. Needs clarification.
         # Let's treat it as hourly for now based on TEMPS_MAXIM=24, but cap firmly at max_cost.
         duration_in_tariff_units = min(duration_minutes / 60.0, max_time)
    
    else: # Unknown unit
        print(f"Warning: Unknown time unit '{unit}' for Tariff ID {tariff_id}.")
        return np.nan

    # --- Calculate Cost based on frac_type ---
    if frac_type == 1: # Rate per unit (usually hour)
        cost = rate * duration_in_tariff_units
    elif frac_type == 2: # Rate per unit (day for RES, event for BUFM?)
        cost = rate * duration_in_tariff_units
    elif frac_type == 0: # Free or fixed rate (DUM, School)
        # Check if rate is non-zero (e.g., EDM2 has rate 2.8)
        if rate > 0 and duration_in_tariff_units > 0: # Apply fixed rate if duration > 0
             cost = rate 
        else: # Otherwise free
             cost = 0.0
        # Note: max_cost might still apply if fixed rate exceeds it? Unlikely here.
    else:
        print(f"Warning: Unknown frac_type '{frac_type}' for Tariff ID {tariff_id}.")
        return np.nan

    # Apply max_cost cap - important! Use np.fmin for float safety?
    cost = min(cost, max_cost)
    
    # Ensure non-negative cost
    cost = max(cost, 0.0)

    # Round to 2 decimal places (cents) - common for currency
    return round(cost, 2)


# --- Example Usage ---
print("\n--- Example Cost Calculations ---")

test_tariffs = {
    'Blue A (TID 46)': 46,
    'Blue D (TID 49)': 49,
    'Resident (TID 52)': 52,
    'DUM (TID 58)': 58,
    'School Free (TID 62 - EDM1)': 62,
    'School Fixed (TID 74 - EDM2)': 74,
    'Motorcycle (TID 53 - MA1)': 53,
    'Bus (TID 64 - BU3)': 64,
    'Event (TID 71 - BUFM)': 71
}
test_durations_minutes = [0, 5, 15, 30, 75, 120, 150, 180, 240, 300, (24*60), (2*24*60), (21*24*60)] # 0m to 21 days

print(f"{'Duration (min)':<15} | {'Tariff':<25} | {'Cost (€)':<8}")
print("-" * 55)

for duration in test_durations_minutes:
    print(f"\n--- Duration: {duration} min ---")
    for name, tid in test_tariffs.items():
        cost = calculate_parking_cost(tid, duration)
        status_str = f"{cost:.2f}" if not np.isnan(cost) else "Error"
        print(f"{duration:<15} | {name:<25} | {status_str:<8}")

print("\n--- End of Cost Calculation Examples ---")


--- Defining Parking Cost Calculation Function ---

--- Example Cost Calculations ---
Duration (min)  | Tariff                    | Cost (€)
-------------------------------------------------------

--- Duration: 0 min ---
0               | Blue A (TID 46)           | 0.00    
0               | Blue D (TID 49)           | 0.00    
0               | Resident (TID 52)         | 0.20    
0               | DUM (TID 58)              | 0.00    
0               | School Free (TID 62 - EDM1) | 0.00    
0               | School Fixed (TID 74 - EDM2) | 0.00    
0               | Motorcycle (TID 53 - MA1) | 0.00    
0               | Bus (TID 64 - BU3)        | 0.00    
0               | Event (TID 71 - BUFM)     | Error   

--- Duration: 5 min ---
5               | Blue A (TID 46)           | 0.21    
5               | Blue D (TID 49)           | 0.09    
5               | Resident (TID 52)         | 0.20    
5               | DUM (TID 58)              | 0.00    
5               | School Free (T